In [ ]:
!pip install tensorflow
!pip install scipy
!pip install Pillow

In [2]:
#import all libraries

import os
import h5py

import numpy as np
from numpy import linalg as LA

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

from scipy.spatial import distance
from PIL import Image



In [3]:
# Understanding the VGG16 Model
model = VGG16(weights = 'imagenet', input_shape = ((224, 224, 3)), pooling = 'max', include_top = False)
model.summary()
# include_top is false since you don't want to include the top layer, which classifies the object which is not needed for image similarity
# the output shape (batch size, height, width, # of channels)

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 512)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
print("hi")
class VGGNet:
    def __init__(self):
        self.input_shape = (224, 224, 3)
        # pre-trained weights
        self.weight = 'imagenet'
        self.pooling = 'max'
        self.model = VGG16(weights = self.weight, input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]), pooling = self.pooling, include_top= False)

        # creates a NumPy array filled entirely with 0s
        # 1 image, 224x224, 3 (RGB channels all initialized to 0 so that pixed is purely black)
        # can think of it like a dummy test case to kind of "warm up" the model to prevent errors later
        # output is never used
        self.model.predict(np.zeros((1,224, 224, 3)))
    

    # output is 512 features in a vector
    def extract_feat(self, img_path):
        # loads image from disk and resizes it to 224x224 so all images are same shape (needed for CNN)
        img = image.load_img(img_path, target_size = (self.input_shape[0], self.input_shape[1]))
        
        # creates an aray of shape (224, 224, 3)
        # for each of the 224*224 pixels, the RGB value (3 sep values) are stored
        img = image.img_to_array(img)

        # axis = 0 --> add new dimension at front
        # turns image into batch with 1 image in it (requires this shape of (batch size, height, width, channels))
        # now image.shape = (1, 224, 224, 3)
        img = np.expand_dims(img, axis = 0)

        # function preprocess_input customized specifically for VGG16 (to match the data VGG16 was originally trained with)
        # 1. reorders channels from RGB --> BGR
        # 2. converts pixel from [0, 255] to float32 (essentially into decimals)
        # 3. centers colors around 0 by subtracting average color values from each pixel (usually around [-128, 128])
        img = preprocess_input(img)

        # returns a 512-dimensional feature vector (NumPy array of shape (1, 12) )
        feat = self.model.predict(img)

        # normalizing the vector by dividing the actual vector by the length/magnitude of vector to get the "unit" vector of length 1
        # doing this so you can fairly compare feature vectors using cosine similarity later
        norm_feat = feat[0]/LA.norm(feat[0])
        
        return norm_feat

hi


In [5]:

images_path = "all images/"

# more concise way to write using list comprehension
#img_list = [os.path.join(images_path, f) for f in os.listdir(images_path)]

# expanded
# buiilds a list of full file paths
img_list = []
for f in os.listdir(images_path):
    full_path = os.path.join(images_path, f)
    img_list.append(full_path)
print(" start feature extraction")

model = VGGNet()

path = "all images/"

feats = []
names = []

for im in os.listdir(path): 
    print("Extracting features from image - ", im)
    X = model.extract_feat(path + im)

    feats.append(X)
    names.append(im)

feat = np.array(feats)

output = "CNNFeatures.h5"

print(" writing feature extraction results to h5 file")

# writing them to hard drive (disk)
h5f = h5py.File(output, 'w')
h5f.create_dataset('dataset_1', data = feats)
h5f.create_dataset('dataset_2', data = np.bytes_(names))
h5f.close()


 start feature extraction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
Extracting features from image -  blue_tank.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Extracting features from image -  pink_tank.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Extracting features from image -  dress.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Extracting features from image -  pants.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
 writing feature extraction results to h5 file


In [6]:
with h5py.File("CNNFeatures.h5", 'r') as h5f:

    # override feats into a new NumPy array containing same data as before but being read from the file
    feats = h5f['dataset_1'][:]
    imgNames = h5f['dataset_2'][:]

# creating feature vecture for query image
queryImg = "query images/pantQuery.png"
query_feat = model.extract_feat(queryImg)

# comparing query image's feature vector to one image vector at a time from feats (being looped through with iteration variable feat)
# cosine distance is outputted from [0, 2] (lower is better)
# similarity = 1 - distance.cosine(a,b) --> range [-1, 1] (higher is better)
# 1 --> most similar, 0 --> no similarity, -1 --> compeltely opposite
scores = [1 - distance.cosine(query_feat, feat) for feat in feats]
scores = np.array(scores)

#np.argsort(scores) returns the indices that would sort an array
# [:: -1] reverse so in descending order
rank_ID = np.argsort(scores)[:: -1]

# creates new array in the order of the indices given by the rank_ID array (already sorted from most similar to  least)
rank_score = scores[rank_ID]

# Get top 3 matches
top_n = 3
top_matches = rank_ID[: top_n]
top_scores = rank_score[:top_n]

print(f"Top {top_n} matches with similarity scores:")
for i, (image_id, score) in enumerate(zip(top_matches, top_scores)):
    # safety check to make sure filenames are stored correctly
    image_name = imgNames[image_id].decode('utf-8') if isinstance(imgNames[image_id], bytes) else imgNames[image_id]
    print(f"{i+1}. Image: {image_name}, Score: {score: .4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Top 3 matches with similarity scores:
1. Image: pants.png, Score:  0.8284
2. Image: dress.png, Score:  0.6029
3. Image: pink_tank.png, Score:  0.5579
